In [2]:
import folium
from folium.plugins import HeatMap
import geopandas as gpd
import pandas as pd
import branca.colormap as cm
import json

mapa = folium.Map(location=[-14.0946, -75.7139], zoom_start=10)

# Cargar datos de robo
data_csv = pd.read_csv("./data/ica_robo.csv")

# Normalizar los datos de robo al rango 0-100
data_csv["Robo"] = round((data_csv["Robo"] / data_csv["Robo"].max()) * 100, 2)

dict_data = data_csv.set_index("Distrito")["Robo"]

# GeoJson
ruta_geojson = "data/distritos-peru.geojson"
with open(ruta_geojson, "r") as archivo_geojson:
    geojson_data = json.load(archivo_geojson)

# Lista de distritos de la provincia de Ica
distritos_ica = data_csv["Distrito"].tolist()

# Filtrando los de la provincia de Ica
filtered_features = []
for feature in geojson_data["features"]:
    if feature["properties"]["nombprov"] == "ICA" and feature["properties"]["nombdist"] in distritos_ica:
        filtered_features.append(feature)

geojson_data["features"] = filtered_features

colormap = cm.LinearColormap(
    [
        "green",
        "yellow",
        "orange",
        "red",
    ],
    vmin=0,
    vmax=100,
)
colormap.caption = "Porcentaje de robo (%)"

# Agregar datos de robo a las propiedades del GeoJSON
for feature in geojson_data["features"]:
    district = feature["properties"]["nombdist"]
    feature["properties"]["Robo"] = dict_data[district]


# Definir la función de estilo
def style_function(feature):
    robbery_rate = feature["properties"]["Robo"]
    return {
        "fillColor": colormap(robbery_rate),
        "color": "black",
        "weight": 2,
        "fillOpacity": 0.7,
    }


# Agregar GeoJson al mapa con el tooltip personalizado
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=["nombdist", "Robo"],
        aliases=["Distrito", "Porcentaje de robo"],
        labels=True,
        sticky=True,
    ),
).add_to(mapa)

mapa.add_child(colormap)
mapa

ModuleNotFoundError: No module named 'folium'